In [1]:
import numpy as np
import scipy.sparse
from scipy.sparse.linalg import lobpcg, eigsh
from scipy.sparse import csr_matrix
import time

In [2]:
adj_path = "/Users/i.lobov/hyperwords/data/wiki/wikipedia.corpus.nodups_counts_win=1.adj"
adjacency_matrix = scipy.sparse.load_npz(adj_path + ".npz")
adjacency_matrix.data = np.ones_like(adjacency_matrix.data, dtype=np.float64)

In [3]:
n = adjacency_matrix.shape[0]
degrees = np.asarray(adjacency_matrix.sum(axis=1)).flatten()
D = scipy.sparse.spdiags(degrees, [0], n, n, format='csr')
L = D - adjacency_matrix
degrees_sqrt = np.sqrt(degrees)
D_inv_sqrt = scipy.sparse.spdiags(1.0 / degrees_sqrt, [0], n, n, format='csr')
L = D_inv_sqrt.dot(L.dot(D_inv_sqrt))

In [4]:
### Initial step
rng = np.random.RandomState(0)
dim = 51
init = rng.rand(n, dim)
init[:, 0] = degrees_sqrt
vals, vecs = lobpcg(L, X=init, maxiter=100, largest=False, verbosityLevel=1)

Solving generalized eigenvalue problem with preconditioning

matrix size 189533
block size 51

No constraints


iteration 0
current block size: 50
eigenvalue: [ -9.67921317e-15   9.98312152e-01   9.98380793e-01   9.98457002e-01
   9.98487424e-01   9.98520206e-01   9.98532051e-01   9.98583051e-01
   9.98644007e-01   9.98681632e-01   9.98688627e-01   9.98721160e-01
   9.98778709e-01   9.98823126e-01   9.98835939e-01   9.98875172e-01
   9.98895839e-01   9.98907764e-01   9.98947802e-01   9.98961920e-01
   9.98973479e-01   9.99031479e-01   9.99076097e-01   9.99101508e-01
   9.99112562e-01   9.99137453e-01   9.99163813e-01   9.99200518e-01
   9.99213717e-01   9.99221323e-01   9.99257599e-01   9.99307719e-01
   9.99321386e-01   9.99364384e-01   9.99383241e-01   9.99405335e-01
   9.99424025e-01   9.99471946e-01   9.99517710e-01   9.99560318e-01
   9.99576117e-01   9.99602373e-01   9.99649524e-01   9.99657233e-01
   9.99697431e-01   9.99742160e-01   9.99768698e-01   9.99795616e-01
   9.99920054

iteration 5
current block size: 46
eigenvalue: [ -9.67921317e-15   5.43916752e-01   6.05717982e-01   6.48894446e-01
   6.68231644e-01   6.84534688e-01   6.95105727e-01   7.11702575e-01
   7.21188043e-01   7.29103190e-01   7.37606639e-01   7.41279521e-01
   7.52174019e-01   7.58741057e-01   7.63617042e-01   7.72940744e-01
   7.77169491e-01   7.78427400e-01   7.82031484e-01   7.84517247e-01
   7.89743870e-01   7.92832871e-01   7.96616723e-01   7.96760176e-01
   8.02011462e-01   8.03841642e-01   8.06380642e-01   8.06891882e-01
   8.08705946e-01   8.09169038e-01   8.10877082e-01   8.11621210e-01
   8.14074078e-01   8.19316225e-01   8.20109597e-01   8.23314287e-01
   8.26903023e-01   8.29411190e-01   8.31947693e-01   8.35354997e-01
   8.36128413e-01   8.38138228e-01   8.40066418e-01   8.42488399e-01
   8.44922915e-01   8.46172231e-01   8.48191760e-01   8.49112111e-01
   8.52760848e-01   8.53839851e-01   8.57967202e-01]
residual norms: [  4.51402916e-13   1.57594516e-03   3.04221479e-03   3.

iteration 10
current block size: 5
eigenvalue: [ -9.67921317e-15   5.43911658e-01   6.05693230e-01   6.48848483e-01
   6.68182777e-01   6.84355498e-01   6.94948223e-01   7.11358221e-01
   7.20768311e-01   7.28554556e-01   7.36702693e-01   7.40450073e-01
   7.50891318e-01   7.56966443e-01   7.62115718e-01   7.70768596e-01
   7.74797037e-01   7.75431930e-01   7.78017225e-01   7.82025832e-01
   7.85380230e-01   7.87665208e-01   7.90503113e-01   7.91734890e-01
   7.92769433e-01   7.97270879e-01   7.98190173e-01   8.00271294e-01
   8.01679225e-01   8.02916446e-01   8.03494006e-01   8.04006381e-01
   8.04811628e-01   8.05144145e-01   8.10413977e-01   8.12496594e-01
   8.14564099e-01   8.15481403e-01   8.16356855e-01   8.19451542e-01
   8.21637441e-01   8.24054531e-01   8.26279162e-01   8.26965953e-01
   8.29452931e-01   8.31043371e-01   8.32688388e-01   8.33674584e-01
   8.34442352e-01   8.34936069e-01   8.35808903e-01]
residual norms: [  4.51478053e-13   5.63883094e-04   9.05304807e-04   9.

In [6]:
from representations.matrix_serializer import load_vocabulary

path = "/Users/i.lobov/hyperwords/data/wiki/wikipedia.corpus.nodups_counts_win=1.words.vocab"
wi, iw = load_vocabulary(path)

path = "/Users/i.lobov/hyperwords/testsets/ws/bruni_men.txt"
test = []
with open(path) as f:
    for line in f:
        x, y, sim = line.strip().lower().split()
        if x in wi and y in wi:
            test.append(((x, y), float(sim)))

In [9]:
from scipy.stats.stats import spearmanr

def evaluate(vecs):
    m = vecs.copy()
    m = m / np.linalg.norm(m, axis=1, keepdims=True)

    def represent(w):
        return m[wi[w], :]

    def similarity(w1, w2):
        return represent(w1).dot(represent(w2))

    results = []
    for (x, y), sim in test:
        results.append((similarity(x, y), sim))

    actual, expected = zip(*results)
    print(spearmanr(actual, expected)[0])

In [10]:
evaluate(vecs)

0.401791812264


In [18]:
ipr = (vecs**4).sum(axis=0)
print("Max ipr is %f" % np.max(ipr))
learning_rate = 10
threshold = 5/n

for i in range(100):
    print("Iteration %d" % i)
    max_ipr_index = np.argmax(ipr)
    update_vec = vecs[:, max_ipr_index]**2 * learning_rate
    update_matrix = scipy.sparse.spdiags(update_vec, [0], n, n, format='csr')
    L = L + update_matrix
    vals, vecs = lobpcg(L, X=vecs, maxiter=100, largest=False, verbosityLevel=1)
    
    ipr = (vecs**4).sum(axis=0)
    print("Max ipr is %f" % np.max(ipr))
    evaluate(vecs)
    if np.max(ipr) < threshold:
        break

Max ipr is 0.386382
Iteration 0
Solving generalized eigenvalue problem with preconditioning

matrix size 189533
block size 51

No constraints


iteration 0
current block size: 5
eigenvalue: [  4.56869595e-07   5.43912021e-01   6.48848441e-01   6.68182859e-01
   6.84358090e-01   6.94949828e-01   7.11358786e-01   7.20768384e-01
   7.28554538e-01   7.36703689e-01   7.40454592e-01   7.50894968e-01
   7.56966567e-01   7.62116349e-01   7.70768320e-01   7.74795812e-01
   7.75430938e-01   7.78015596e-01   7.82025165e-01   7.85377704e-01
   7.87663254e-01   7.90501868e-01   7.91728524e-01   7.92767487e-01
   7.97268270e-01   7.98186860e-01   8.00252945e-01   8.01669612e-01
   8.02912615e-01   8.03483280e-01   8.04000840e-01   8.04806212e-01
   8.05136539e-01   8.10407928e-01   8.12459272e-01   8.14534108e-01
   8.15349353e-01   8.16322943e-01   8.19443245e-01   8.21630714e-01
   8.24029543e-01   8.26255064e-01   8.26934146e-01   8.29256928e-01
   8.31021101e-01   8.31626929e-01   8.31983810e-01

iteration 5
current block size: 1
eigenvalue: [  4.26730360e-07   5.43911998e-01   6.48848313e-01   6.68182806e-01
   6.84357521e-01   6.94949224e-01   7.11358501e-01   7.20768340e-01
   7.28554078e-01   7.36703253e-01   7.40452727e-01   7.50893599e-01
   7.56966488e-01   7.62116160e-01   7.70767762e-01   7.74795388e-01
   7.75430580e-01   7.78015010e-01   7.82024349e-01   7.85375904e-01
   7.87662991e-01   7.90500788e-01   7.91725701e-01   7.92766304e-01
   7.97264179e-01   7.98185949e-01   8.00244989e-01   8.01662534e-01
   8.02912254e-01   8.03482451e-01   8.03997952e-01   8.04804022e-01
   8.05134178e-01   8.10407348e-01   8.12456243e-01   8.14525143e-01
   8.15343838e-01   8.16319097e-01   8.19441412e-01   8.21617320e-01
   8.23972099e-01   8.26239868e-01   8.26930175e-01   8.28973344e-01
   8.30979109e-01   8.31395968e-01   8.31901854e-01   8.33485156e-01
   8.34292360e-01   8.34429862e-01   8.66574042e-01]
residual norms: [  7.93160310e-05   5.66601920e-04   9.48909987e-04   1.2

iteration 10
current block size: 1
eigenvalue: [  4.26382597e-07   5.43911997e-01   6.48848310e-01   6.68182805e-01
   6.84357482e-01   6.94949204e-01   7.11358445e-01   7.20768333e-01
   7.28553994e-01   7.36703235e-01   7.40452413e-01   7.50893526e-01
   7.56966463e-01   7.62116119e-01   7.70767758e-01   7.74795328e-01
   7.75430432e-01   7.78014952e-01   7.82024080e-01   7.85375452e-01
   7.87662954e-01   7.90500572e-01   7.91725110e-01   7.92765626e-01
   7.97263189e-01   7.98185890e-01   8.00242609e-01   8.01661649e-01
   8.02911838e-01   8.03482411e-01   8.03995826e-01   8.04803755e-01
   8.05131599e-01   8.10403758e-01   8.12449471e-01   8.14522945e-01
   8.15343134e-01   8.16316757e-01   8.19440503e-01   8.21609651e-01
   8.23956759e-01   8.26228692e-01   8.26924220e-01   8.28827800e-01
   8.30952751e-01   8.31271062e-01   8.31895866e-01   8.33445156e-01
   8.34206330e-01   8.34407298e-01   8.43589521e-01]
residual norms: [  7.29041234e-05   5.65865118e-04   9.48949036e-04   1.

iteration 15
current block size: 1
eigenvalue: [  4.26275413e-07   5.43911996e-01   6.48848307e-01   6.68182796e-01
   6.84357465e-01   6.94949188e-01   7.11358413e-01   7.20768324e-01
   7.28553977e-01   7.36703222e-01   7.40452403e-01   7.50893513e-01
   7.56966437e-01   7.62116116e-01   7.70767739e-01   7.74795327e-01
   7.75430311e-01   7.78014715e-01   7.82023924e-01   7.85375432e-01
   7.87662904e-01   7.90500483e-01   7.91724004e-01   7.92764711e-01
   7.97263117e-01   7.98185867e-01   8.00242100e-01   8.01661354e-01
   8.02910799e-01   8.03482204e-01   8.03995084e-01   8.04803594e-01
   8.05130933e-01   8.10402506e-01   8.12447800e-01   8.14520423e-01
   8.15342940e-01   8.16314569e-01   8.19439237e-01   8.21607383e-01
   8.23942453e-01   8.26220685e-01   8.26922548e-01   8.28741257e-01
   8.30918276e-01   8.31190356e-01   8.31893370e-01   8.33428284e-01
   8.34137835e-01   8.34393703e-01   8.38390317e-01]
residual norms: [  7.16576478e-05   5.64743519e-04   9.47494130e-04   1.

iteration 20
current block size: 1
eigenvalue: [  4.26213710e-07   5.43911995e-01   6.48848306e-01   6.68182794e-01
   6.84357453e-01   6.94949177e-01   7.11358402e-01   7.20768319e-01
   7.28553975e-01   7.36703215e-01   7.40452395e-01   7.50893464e-01
   7.56966367e-01   7.62116013e-01   7.70767738e-01   7.74795325e-01
   7.75430270e-01   7.78014449e-01   7.82023914e-01   7.85375344e-01
   7.87662867e-01   7.90500342e-01   7.91722338e-01   7.92764057e-01
   7.97262878e-01   7.98185843e-01   8.00241266e-01   8.01661171e-01
   8.02909425e-01   8.03481966e-01   8.03995078e-01   8.04803568e-01
   8.05130422e-01   8.10401603e-01   8.12447766e-01   8.14519272e-01
   8.15342802e-01   8.16313345e-01   8.19438023e-01   8.21606012e-01
   8.23934883e-01   8.26213792e-01   8.26920662e-01   8.28705511e-01
   8.30891159e-01   8.31134776e-01   8.31885198e-01   8.33426593e-01
   8.34058378e-01   8.34372940e-01   8.36318193e-01]
residual norms: [  7.11079060e-05   5.64383977e-04   9.47131482e-04   1.

...

iteration 10
current block size: 1
eigenvalue: [ 0.00329277  0.54639573  0.66635808  0.68764614  0.70092566  0.71812395
  0.72772753  0.74106009  0.74344038  0.7450566   0.75307091  0.76179236
  0.76745084  0.77532801  0.7798674   0.78182245  0.78765641  0.79111237
  0.79361121  0.79740632  0.80142322  0.8032183   0.80568     0.80682064
  0.80940879  0.81286769  0.81398046  0.81545843  0.81880182  0.8206412
  0.82271782  0.82541885  0.82698678  0.8295827   0.83136366  0.83216928
  0.83516066  0.83622084  0.83803637  0.83837165  0.83910038  0.84198226
  0.84377671  0.84614601  0.84626578  0.84889197  0.84962001  0.85107478
  0.85300323  0.85399352  0.85520076]
residual norms: [ 0.00137916  0.00327619  0.00509436  0.00221242  0.00386105  0.00265612
  0.00242664  0.00228867  0.00344728  0.00279576  0.00324266  0.00400933
  0.00457551  0.00329213  0.00240744  0.00463571  0.00355897  0.00357375
  0.00385239  0.00388912  0.00472878  0.00443107  0.00472805  0.00287526
  0.00543262  0.00475725

iteration 17
current block size: 1
eigenvalue: [ 0.00329277  0.54639568  0.66635797  0.68764612  0.70092565  0.71812381
  0.72772753  0.74106005  0.74344027  0.74505655  0.75307084  0.76179217
  0.76745072  0.775328    0.77986734  0.78182238  0.7876563   0.79111217
  0.79361118  0.79740613  0.80142309  0.80321827  0.80567993  0.80682039
  0.80940813  0.8128663   0.81398024  0.81545834  0.8188013   0.8206404
  0.82271736  0.82541868  0.82698651  0.82958216  0.83136359  0.83216897
  0.83516032  0.83622067  0.83803625  0.83837073  0.83909927  0.84198183
  0.84377567  0.84614543  0.84626322  0.84888736  0.84961913  0.85106487
  0.8529487   0.85366681  0.85518787]
residual norms: [ 0.00137698  0.00326534  0.00507774  0.00220784  0.00386077  0.00260599
  0.00242554  0.00226702  0.00342573  0.00278928  0.00323772  0.00398182
  0.00457283  0.00328895  0.00240702  0.00463306  0.00353544  0.00352429
  0.00384361  0.00387993  0.0047115   0.00442188  0.00471377  0.00279966
  0.00537553  0.00444694

iteration 5
final eigenvalue: [ 0.00351464  0.54659424  0.66667163  0.68778712  0.70106015  0.71836299
  0.72782976  0.74120032  0.74364315  0.74520932  0.75318441  0.76197538
  0.76770622  0.77580422  0.78005375  0.78446912  0.78788596  0.7913229
  0.79384832  0.8008474   0.80330815  0.80361694  0.80599283  0.8073729
  0.81028732  0.81393067  0.81571312  0.81669086  0.81921327  0.82102989
  0.82290671  0.82595284  0.8272887   0.83016555  0.83159661  0.83235895
  0.83530012  0.83638149  0.83827523  0.83851983  0.83946257  0.84212843
  0.8439498   0.84627827  0.84643512  0.84903163  0.84973597  0.85131323
  0.85311147  0.85379849  0.85534104]
final residual norms: [ 0.00130109  0.00331254  0.00470427  0.00210541  0.00386577  0.00246429
  0.00247053  0.0022141   0.00319892  0.00281842  0.00329818  0.00366457
  0.00467934  0.00348967  0.00248135  0.00356921  0.00363078  0.00329195
  0.00383119  0.00428405  0.00493376  0.00417203  0.00473346  0.00258652
  0.00437977  0.00426791  0.0052075 

iteration 1
current block size: 4
eigenvalue: [ 0.00373225  0.54679382  0.66708258  0.68799368  0.70136012  0.71866459
  0.72804676  0.74145068  0.74392751  0.74542107  0.75337576  0.7622098
  0.76827278  0.77620683  0.78025704  0.78500703  0.78822598  0.79154913
  0.79431403  0.80150998  0.80392328  0.80499338  0.80685995  0.80793032
  0.81171162  0.81589044  0.81853696  0.81993097  0.82187614  0.82331344
  0.82692745  0.82919353  0.83107384  0.83223537  0.8328111   0.83524259
  0.83612901  0.83664957  0.83877084  0.83910234  0.83986965  0.84253505
  0.84428294  0.84650071  0.84677148  0.84927764  0.85008732  0.85156615
  0.8533668   0.85400406  0.85565437]
residual norms: [ 0.00137972  0.0034579   0.0045613   0.0021492   0.00421092  0.00281246
  0.00264027  0.00204431  0.00318574  0.0030686   0.00357377  0.00341053
  0.0052394   0.00417501  0.00273352  0.00356358  0.00387712  0.00295427
  0.00435816  0.00522553  0.00551653  0.00557051  0.00499371  0.00292534
  0.00483866  0.00501503 

KeyboardInterrupt: 

In [19]:
output_path = "../data/wiki/win=1_x-laplacin_pow=0.00_dim=50"
np.save(output_path + ".vecs", vecs)
np.save(output_path + ".vals", vals)
np.save(output_path + ".degrees", degrees)